Decompose crepe data with LLMs
+ Based on Zhipu api.

In [28]:
import csv
from zhipuai import ZhipuAI
import datetime

# zhipu api key
my_api_key = "69a629a0f91455a08236a8dc5d74198b.0XT7BFKna9YEnJSP"

In [ ]:
prompt_training = "你是一个查询语句分解助手，请并结合语义，将Q中的长句，分解成中符合语义的子句，其中每个子句物品的位置信息词组； \
分解的要求是，根据语义，使得分解后的每个子句都是包含原句信息的关键词组；\
如果原句中有多个“,”，子句的名词要根据上下文语义，带有其与前后物品之间的联系；\
以下是Q&A一一配对的示例，示例中将Q中的长句，分解成A中符合语义的子查询\
Q: \"keyboard, computer monitor, printer, and fax machine on a desk, with a chair against the wall.\", \
A: \"keyboard on a desk, computer monitor on a desk, printer on a desk, fax machine on a desk, a chair against the wall\", \
Q: \"woman wearing a sweater with a wrist pad on her keyboard in front of a monitor. there is a sticky note taped to the monitor and a juice bottle next to it.\", \
A: \"woman wearing a sweater, a wrist pad on her keyboard, her keyboard in front of a monitor, a sticky note taped to the monitor, a juice bottle next to monitor.\", \
Q: \"man with sleeves and a hand on a keyboard in front of a cpu. there are books on the cpu and a mouse next to the keyboard.\", \
A: \"man with sleeves, a hand on a keyboard, a keyboard in front of a cpu, books on the cpu, a mouse next to the keyboard.\", \
Q: \"shelves with books and a baby on them, against a wall with a corkboard and posters\", \
A: \"shelves with books, a baby on books, a bady against a wall, a wall with a corkboard, a wall with posters\", \
Q: \"computer on a desk with a mouse and cup. there is a computer tower below the desk and lint on the floor.\", \
A: \"computer on a desk, a desk with a mouse, a desk with a cup, a computer tower below the desk,  and a computer tower below the lint, lint on the floor.\", \
Q: \"a rolodex, desk, chair, monitor, and book on a desk. there is a picture of a baby propped up against the monitor.\", \
A: \"a rolodex on a desk, chair on a desk, monitor on a desk, book on a desk, a picture of a baby propped up against the monitor.\", \
Q: \"envelope, soda cup, book, and pen on a desk next to a computer monitor. there is a picture on the desk.\", \
A: \"envelope on a desk, soda cup on a desk, book on a desk, pen on a desk, a desk next to a computer monitor, a picture on the desk.\", \
Q: \"sign on a pole next to a man on a sidewalk. there is a parking meter and people on the sidewalk. there is a boy by the parking meter.\", \
A: \"sign on a pole, a pole next to a man, a man on a sidewalk, a parking meter on the sidewalk, people on the sidewalk, a boy by the parking meter.\", \
Q: \"keyboard and telephone on top of a desk made of books with a computer and mouse next to it. there is a bookshelf with books on it.\", \
A: \"keyboard on top of a desk, telephone on top of a desk, a desk made of books, books with a computer, books with mouse. a bookshelf with books\", \
Q: \"a man with two eyes and a torso. the man is standing in a room and has a hand.\", \
A: \"a man with two eyes, a man with a torso, a man is standing in a room, a man has a hand.\", \
Q: \"sign on a building with a window and balcony. the door has a frame around it, and there is a second window on the building.\", \
A: \"sign on a building, a building with a window, a building with balcony, the door has a frame around it, a second window on the building.\", \
Q: \"light on a car in front of a house. the car has a left brake-light, and the house has a window. there is an emblem on the back of the car, and a log on the house.\", \
A: \"light on a car, a car in front of a house, the car has a left brake-light, the house has a window, an emblem on the back of the car, a log on the house.\", \
Q: \"there is a tree, another tree, and another tree on the lawn. there is a person hunched over on the lawn, and a light pole in front of the building.\", \
A: \"a tree on the lawn, another tree on the lawn, another tree on the lawn, a person hunched over on the lawn, a light pole in front of the building.\", \
Q: \"a recliner facing a television that is next to a wall. the television is on a stand in the corner and has a sign atop it.\", \
A: \"a recliner facing a television, a television is next to a wall, the television is on a stand, a stand in the corner, the television has a sign atop it.\", \
"

print(prompt_training)

In [30]:
# Small scale test

# prompt_inferring_test = '请根据以上的要求和示例，请给出以下Q的分解结果。输出格式：对于每一个Q，只输出一行分解后的子句 "A=分解后的句子"，不需要输出原句Q，子句之间以","分割，每两个A之间不要有空行。\
#     如果待分解句子中含有it和them等代词，则在分解的子句中，要结合句子含义将他们转换成对应的单词或词组；介词短语不能省略；\
#     每条分解后的子句都是有实际含义短语或短句，短句之间可以有单词相互重复，但尽量包含前后联系的语义信息，不是一个单独的词'

prompt_inferring_test = 'Q&A示例展示完毕，请给出以下Q的分解结果；输出格式：对于每一个Q，各回复一行分解后的子句"A=分解后的句子"，不要输出原句Q，每两行之间不要有空行。'

# input for testing
Q7 = "Q = \"table with photos, a vase, and a bow on it, and a chair by the table\" "
Q8 = "Q = \"lamp stand with a desk lamp on it and duct tape. there are books on a desk and a keyboard on the table.\""
Q9 = "Q = \"a fence surrounding a building, with a bird fountain in front of it that is covered in snow, and a gate leading into the building	\""
Q10 = "Q = \"airplane with wheels in the middle of it and light under it, in the sky with clouds\""

# prompt_inferring = 'Give the answer : Q: "%s", Q: "%s" (only give the A of each Q divided by &&)' % (Q5, Q6)
prompt_inferring_test = prompt_inferring_test + Q7 + Q8 + Q9 + Q10

prompt_test = prompt_training + prompt_inferring_test
print(prompt_test)

Q&A示例展示完毕，请给出以下Q的分解结果；输出格式：对于每一个Q，各回复一行分解后的子句"A=分解后的句子"，不要输出原句Q，每两行之间不要有空行。


In [36]:
# Small scale test

client = ZhipuAI(api_key=my_api_key) # APIKey

# prompt = input("user:")

test_txt_file = 'test_prompt.txt'

while True:

    response = client.chat.completions.create(
        model="glm-4",  # model
        messages=[
            {"role": "user", "content": prompt_test}
        ],
        temperature = 0.3,
    )
    # answer = response.choices[0].message
    answer = response.choices[0].message.content
    print(answer)

    with open(test_txt_file, 'a') as ans_file:
        current_datetime = datetime.datetime.now()
        time_str = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        ans_file.write("\n")
        ans_file.write(time_str)
        ans_file.write("\n")
        ans_file.write(answer)
        ans_file.write("\n")
    
    break

A=table with photos, a vase on a table, a bow on a table, a chair by the table

A=lamp stand with a desk lamp, duct tape on a lamp stand, books on a desk, a keyboard on the table

A=a fence surrounding a building, a bird fountain in front of the building, a fountain covered in snow, a gate leading into the building

A=airplane with wheels in the middle, light under the airplane, the airplane in the sky, clouds in the sky


In [4]:
# for large-scale generation
 
client = ZhipuAI(api_key=my_api_key) # APIKey

input_csv = '.\input\data_input1.csv'
output_txt = 'ans.txt'

# prompt = input("user:")
batch_size = 20
generate_size = 800

batch_begin = -20
batch_end = 0

while True:
    batch_begin = batch_begin + batch_size
    batch_end = batch_begin + batch_size
    if batch_begin >= generate_size:
        break

    print(batch_begin)
    print(batch_end)

    infer = []

    with open(input_csv, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for i, row in enumerate(csv_reader):
            if i >= batch_begin and i < batch_end:
                infer.append(row[0])
            # infer.append(row[0])

    # print(infer)

    prompt_inferring = 'Q&A示例展示完毕，请给出以下Q的分解结果；输出格式：对于每一个"Q:"的原长句，返回一行分解后的子句，你不应该输出原句Q；\
     如果待分解句子中含有it，则在分解的子句中，要结合句子含义将it转换成对应的名词；分解后的句子中冠词a不省略。'
    for i in infer:
        prompt_inferring = prompt_inferring + "Q: \"" + i + "\","

    # print(prompt_inferring)

    prompt = prompt_training + prompt_inferring
    # print(prompt)


    response = client.chat.completions.create(
        model="glm-4",  # model
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature = 0.3,
    )
    # answer = response.choices[0].message
    answer = response.choices[0].message.content
    print(answer)

    with open(output_txt, 'a') as ans_file:
        ans_file.write(answer)
        ans_file.write("\n")
    

260
280
white bin on right side of playroom floor, pink and white table to right of playroom, colorful rug in middle of playroom, blue post to right of playroom, small white fridge in back of playroom.
bread and debris on a table, a chair at the top of the table.
three cabinets in a kitchen, a blender and bottle on the counter.
utensils next to a can, a clock display with a blender on top, a cabinet above the clock display.
cabinet with a knob and a microwave oven, a stove with a clock display and a knob on the back, a can sitting on the clock display.
blender sitting on a clock display, a stove with two knobs on the back and an oven door with a glass window.
stainless steel jar with utensils, sink with a drying rack, faucet on the sink.
coffee maker in corner, toaster on the counter, bag of coffee on the counter, four knives on the counter, cabinets above the counter.
a stove with a knob, cooking pot on the stove, two drawers below the stove, three knobs on the stove.
a pan on a stove